In [51]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import sklearn

In [52]:
## Load the trained model.h5, scaler pickle, onehot picke

model=load_model('model.h5')

##Load encoders and scalers

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)






In [53]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [54]:
geo_encoded=onehot.transform([[input_data['Geography']]])
print(geo_encoded)
geo_df=pd.DataFrame(geo_encoded,columns=onehot.get_feature_names_out(['Geography']))
geo_df

[[1. 0. 0.]]


c:\Users\Harjeet Singh Pannu\Desktop\ANN project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [55]:
input_df=pd.DataFrame([input_data]) ## Converting dictionary to dataframe (tabular)
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [56]:

input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [57]:
# 🧠 Why we are NOT doing this for OneHotEncoder:
# input_df['Geography'] = onehot.transform(...) ❌
# ❌ Because:
# - onehot.transform() returns a 2D array → e.g., [[0. 0. 1.]]
# - But input_df['Geography'] is a 1D column → it expects only one value per row
# - Trying to assign multiple values into a single column causes shape mismatch → ERROR!
# - So, we can't overwrite the 'Geography' column directly like this.

# ✅ What we do instead for OneHotEncoder:
# - Step 1: Drop the original 'Geography' column from input_df (it has raw text like 'India', 'France')
# - Step 2: geo_df is a DataFrame with new columns from the one-hot encoded values
#     e.g., 'Geography_France', 'Geography_Germany', 'Geography_India' (each is 0 or 1)
# - Step 3: Combine the updated input_df and geo_df horizontally using pd.concat(axis=1)
#     This way, we preserve all the data, replace 'Geography' with its numeric version, and make the DataFrame ML-ready

input_df = pd.concat([input_df.drop("Geography", axis=1), geo_df], axis=1)

# ✅ For LabelEncoder, direct assignment is OK:
# LabelEncoder gives back a 1D array (just one value per row, like 0, 1, or 2)
# So we can directly assign it back to a single column without any issues
# Example:
# If label_encoder_gender is already fitted, we can do:

# 🎯 Summary:
# - LabelEncoder = 1D → direct assignment works ✅
# - OneHotEncoder = 2D → use concat after dropping the original column ✅

In [58]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [59]:
## Scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [60]:
## Predict churn

prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 55ms/step


array([[0.03710573]], dtype=float32)

In [61]:
pred_prob=prediction[0][0] ## multidimensional array
pred_prob

0.03710573

In [62]:
if pred_prob>0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn


In [63]:
# ❓ Is Customer Churn Prediction the same as Sentiment Analysis?
# 🧠 Simple Answer: ❌ No, they are two very different things.

# --------------------------------------------------------------------------------
# 🔍 What is Customer Churn Prediction?
# --------------------------------------------------------------------------------
# 👉 "Churn" means a customer decides to leave or stop using a company's service.
# 🚀 The goal here is to PREDICT whether a customer will stay or leave.

# ✅ It is a MACHINE LEARNING classification problem.
# ✅ We mostly use numerical or tabular data (not text).
# Examples of input features:
# - Gender
# - Age
# - Monthly charges
# - Type of contract (e.g., prepaid, postpaid)
# - Number of complaints
# - Tenure (how long the customer has stayed)
# ➡️ Based on these, the model predicts:
#    - 0 = Customer stays
#    - 1 = Customer leaves (churns)

# 🛠️ Techniques used: Logistic Regression, Decision Trees, Random Forest, Neural Networks etc.

# --------------------------------------------------------------------------------
# 🔍 What is Sentiment Analysis?
# --------------------------------------------------------------------------------
# 👉 This is about detecting emotions or opinions from TEXT.
# ✅ It is a Natural Language Processing (NLP) task.
# ✅ Used to understand if a person is:
#    - Happy (positive sentiment)
#    - Angry or sad (negative sentiment)
#    - Neutral (no strong emotion)
# 🔡 Input data is in the form of:
#    - Tweets
#    - Reviews
#    - Feedback messages
#    - Comments

# ➡️ The output is usually:
#    - "Positive"
#    - "Negative"
#    - "Neutral"

# 🛠️ Techniques used: Bag of Words, TF-IDF, Word Embeddings + classifiers

# --------------------------------------------------------------------------------
# 🧠 Summary Table:
# --------------------------------------------------------------------------------
# | Task                     | Input Type   | Goal                          | Field               |
# |--------------------------|--------------|-------------------------------|---------------------|
# | Customer Churn Prediction| Tabular Data | Predict if customer will leave| Machine Learning    |
# | Sentiment Analysis       | Text Data    | Detect opinion/emotion        | NLP (Text Analysis) |

# --------------------------------------------------------------------------------
# ✅ Bonus Insight:
# You CAN combine both — for example:
# If a company receives a lot of negative complaints in text,
# You could use Sentiment Analysis to extract negativity,
# Then feed that into a Churn Prediction model to improve its accuracy.
